<a href="https://colab.research.google.com/github/MohanVarmaDhana/Dhanamo-AI/blob/main/Semantic%20search%20engine%20using%20Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# Function to query Europe PMC API
def query_europe_pmc(query, max_results=5):
    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    params = {
        "query": query,
        "resultType": "core",  # 'core' gives the main information like title, authors, etc.
        "pageSize": max_results,
        "format": "json"  # Get data in JSON format
    }

    # Send GET request
    response = requests.get(base_url, params=params)
    data = response.json()
    print(data)

    # Check if results are returned
    if data['hitCount'] > 0:
        for i, doc in enumerate(data['resultList']['result']):
            print(f"Paper {i+1}:")
            print(f"Title: {doc.get('title', 'N/A')}")
           # print(f"Authors: {', '.join([author['name'] for author in doc.get('authorList', {}).get('author', [])]) if 'authorList' in doc else 'N/A'}")
            print(f"Abstract: {doc.get('abstractText', 'N/A')}")

            # Check for full-text URL
            full_text_url = doc.get('fullTextUrl', None)
            if full_text_url:
                print(f"Full Text URL: {full_text_url}")
                download_paper(full_text_url)
            else:
                print("Full text not available.")

            print("-" * 80)
    else:
        print("No results found.")

# Function to download the full paper
def download_paper(url):
    try:
        # Check if the full text URL is a PDF or HTML link
        if url.endswith('.pdf'):
            response = requests.get(url)
            filename = url.split('/')[-1]  # Get filename from URL
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded paper as {filename}")
        elif url.endswith('.html'):
            response = requests.get(url)
            filename = "paper.html"
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(response.text)
            print(f"Downloaded paper as {filename}")
        else:
            print("Unsupported file type.")
    except Exception as e:
        print(f"Failed to download the paper: {e}")

# Query example
query_europe_pmc("large language models in pharma", max_results=1)


{'version': '6.9', 'hitCount': 7756, 'nextCursorMark': 'AoIIQJn5fSg1MjY0MjM2Ng==', 'nextPageUrl': 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=large language models in pharma&cursorMark=AoIIQJn5fSg1MjY0MjM2Ng==&resultType=core&pageSize=1&format=json', 'request': {'queryString': 'large language models in pharma', 'resultType': 'core', 'cursorMark': '*', 'pageSize': 1, 'sort': '', 'synonym': False}, 'resultList': {'result': [{'id': '40112233', 'source': 'MED', 'pmid': '40112233', 'pmcid': 'PMC11949217', 'fullTextIdList': {'fullTextId': ['PMC11949217']}, 'doi': '10.1200/cci-24-00230', 'title': 'Large Language Models as Decision-Making Tools in Oncology: Comparing Artificial Intelligence Suggestions and Expert Recommendations.', 'authorString': 'Ah-Thiane L, Heudel PE, Campone M, Robert M, Brillaud-Meflah V, Rousseau C, Le Blanc-Onfroy M, Tomaszewski F, Supiot S, Perennec T, Mervoyer A, Frenel JS.', 'authorList': {'author': [{'fullName': 'Ah-Thiane L', 'firstName': 'Loic'

In [ ]:
!nslookup api.europepmc.org

Server:		127.0.0.11
Address:	127.0.0.11#53

** server can't find api.europepmc.org: NXDOMAIN



In [4]:
import requests

# Function to query Europe PMC API
def query_europe_pmc(query, max_results=5):
    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    params = {
        "query": query,
        "resultType": "core",  # 'core' gives the main information like title, authors, etc.
        "pageSize": max_results,
        "format": "json"  # Get data in JSON format
    }

    # Send GET request
    response = requests.get(base_url, params=params)
    data = response.json()

    # Check if results are returned
    if data['hitCount'] > 0:
        for i, doc in enumerate(data['resultList']['result']):
            print(f"Paper {i+1}:")
            print(f"Title: {doc.get('title', 'N/A')}")
            #print(f"Authors: {', '.join([author['name'] for author in doc.get('authorList', {}).get('author', [])]) if 'authorList' in doc else 'N/A'}")
            #print(f"Abstract: {doc.get('abstractText', 'N/A')}")

            # Extract DOI or PMID
            pmid = doc.get('pmid', None)
            doi = doc.get('doi', None)

            # Use PMID or DOI to get full text
            if pmid:
                print(f"PMID: {pmid}")
                fetch_full_text(pmid)
            elif doi:
                print(f"DOI: {doi}")
                fetch_full_text_by_doi(doi)
            else:
                print("No full text ID available.")
            print("-" * 80)
    else:
        print("No results found.")

# Function to fetch full text using PMID
def fetch_full_text(pmid):
    # Construct the full-text URL using PMID
    pmid = 'PMC'+pmid
    full_text_url = f"https://europepmc.org/articles/{pmid}?pdf=render"
    download_paper(full_text_url)

# Function to fetch full text using DOI
def fetch_full_text_by_doi(doi):
    # Construct the full-text URL using DOI (Europe PMC often redirects to publisher)
    full_text_url = f"https://doi.org/{doi}"
    download_paper(full_text_url)

# Function to download the paper
def download_paper(article_url):
    save_as= article_url.split('/')[-1].split('?')[0]+'.pdf'
    try:
        # Set headers to mimic a browser (sometimes necessary)
        headers = {
            "User-Agent": "Mozilla/5.0"
        }

        # Send request
        response = requests.get(article_url, headers=headers)

        # Check if successful
        if response.status_code == 200:
            with open(save_as, "wb") as f:
                f.write(response.content)
            print(f"PDF downloaded successfully and saved as '{save_as}'")
        else:
            print(f"Failed to download PDF. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error: {e}")

# Query example
query_europe_pmc("Marketing in Pharma or Healthcare", max_results=20)


Paper 1:
Title: Safety of iGlarLixi in Japanese People with Type 2 Diabetes: A Post-marketing Database Study.
PMID: 40056372
Failed to download PDF. Status code: 404
--------------------------------------------------------------------------------
Paper 2:
Title: Industry news update: December 2023
No full text ID available.
--------------------------------------------------------------------------------
Paper 3:
Title: Harmful Marketing: An Overlooked Social Determinant of Health.
PMID: 39789304
Failed to download PDF. Status code: 404
--------------------------------------------------------------------------------
Paper 4:
Title: Pharmaceutical industry use of key opinion leaders to market prescription opioids: A review of internal industry documents.
PMID: 39687444
Failed to download PDF. Status code: 404
--------------------------------------------------------------------------------
Paper 5:
Title: Post-marketing Experience with Cenobamate in the Treatment of Focal Epilepsies: A Mu

In [ ]:
https://europepmc.org/articles/40112233

In [5]:
def download_paper(article_url, save_as="paper.pdf"):
    try:
        # Set headers to mimic a browser (sometimes necessary)
        headers = {
            "User-Agent": "Mozilla/5.0"
        }

        # Send request
        response = requests.get(article_url, headers=headers)

        # Check if successful
        if response.status_code == 200:
            with open(save_as, "wb") as f:
                f.write(response.content)
            print(f"PDF downloaded successfully and saved as '{save_as}'")
        else:
            print(f"Failed to download PDF. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error: {e}")

# Example usage
download_paper("https://europepmc.org/articles/PMC11949217?pdf=render", "PMC11949217.pdf")

PDF downloaded successfully and saved as 'PMC11949217.pdf'


In [ ]:
"https://europepmc.org/articles/{pmid}?pdf=render"